In [1]:
!mkdir -p {SAVE_FOLDER}
!pip install pydub
!pip install pytorch==2.0.0 torchaudio==2.0.0 pytorch-cuda=11.8 -c pytorch -c nvidia
!pip install whisperx
!apt-get install libcudnn8 #VAJNO ZA DA NE KRASHWA

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcudnn8 is already the newest version (8.9.7.29-1+cuda12.2).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [2]:
import torch
print(torch.cuda.is_available())  # Must return True, either we've got a problem
print(torch.version.cuda)

True
12.4


In [3]:
import whisperx
device = "cuda"
audio_file = "inpt.wav"
batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)
model = whisperx.load_model("large-v3", device, compute_type=compute_type)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../usr/local/lib/python3.11/dist-packages/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu124. Bad things might happen unless you revert torch to 1.x.


In [5]:
audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"]) # before alignment

/usr/local/lib/python3.11/dist-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(


Detected language: bg (0.98) in first 30s of audio...
[{'text': ' Здравейте, гледате лице в лице. Продължаваме промяната. Демократична България решиха да останат в опозиция на кабинета Желязков. Защо? Ще чуем отговорите на съпредседателя на Продължаваме промяната, Кирил Петков. Здравейте, господин Петков. Здравейте. Защо избрахте ролята на опозиция? Опозиция на какво сте?', 'start': 6.275, 'end': 25.799}, {'text': ' Ами опозиция сме на модела Пеевски. И го казахме от самото начало с тази декларация, с условията, които, казвам модела, не човекът. Корупционните модели в държавата, изразходването на средства по нелегитимен начин, нелегитимните влияния в съдебната власт. Това е един такъв модел, който аз вярвам, че трябва рано или късно, бавно дори, ако трябва, но да бъде разграждан.', 'start': 27.588, 'end': 55.499}, {'text': ' Добре, защото позиция на кабинета Желязков. Виждате там всичко това, което изброихте преди, маничко ли?', 'start': 55.499, 'end': 62.249}, {'text': ' И знаете ли, 

In [6]:
align_model = "infinitejoy/wav2vec2-large-xls-r-300m-bulgarian"  # Change if you find a better model
l_a, metadata = whisperx.load_align_model(language_code=result["language"],
                                          device=device,
                                          model_name=align_model)

result = whisperx.align(result["segments"], l_a, metadata, audio, device, return_char_alignments=False)

print(result["segments"])  # after alignment


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

[{'start': 6.295, 'end': 8.137, 'text': ' Здравейте, гледате лице в лице.', 'words': [{'word': 'Здравейте,', 'start': 6.295, 'end': 6.936, 'score': 0.249}, {'word': 'гледате', 'start': 6.976, 'end': 7.356, 'score': 0.238}, {'word': 'лице', 'start': 7.396, 'end': 7.657, 'score': 0.352}, {'word': 'в', 'start': 7.677, 'end': 7.717, 'score': 0.062}, {'word': 'лице.', 'start': 7.737, 'end': 8.137, 'score': 0.158}]}, {'start': 8.758, 'end': 9.98, 'text': 'Продължаваме промяната.', 'words': [{'word': 'Продължаваме', 'start': 8.758, 'end': 9.359, 'score': 0.458}, {'word': 'промяната.', 'start': 9.419, 'end': 9.98, 'score': 0.39}]}, {'start': 10.02, 'end': 15.787, 'text': 'Демократична България решиха да останат в опозиция на кабинета Желязков.', 'words': [{'word': 'Демократична', 'start': 10.02, 'end': 10.66, 'score': 0.408}, {'word': 'България', 'start': 10.68, 'end': 11.241, 'score': 0.236}, {'word': 'решиха', 'start': 11.401, 'end': 11.882, 'score': 0.305}, {'word': 'да', 'start': 11.942, '

In [9]:
from pyannote.audio.pipelines import SpeakerDiarization
from pyannote.audio import Model, Inference
from huggingface_hub import login
from google.colab import userdata
HF_SECRET = userdata.get('HF_SECRET')

login(token=HF_SECRET)  # Authenticate HF_SECRET is my huggingface API key

In [10]:
# 3. Assign speaker labels
diarize_model = whisperx.DiarizationPipeline(device=device, )

# add min/max number of speakers if known
diarize_segments = diarize_model(audio, min_speakers=2, max_speakers=2)

# diarize_segments = diarize_model(audio
# diarize_model(audio, min_speakers=min_speakers, max_speakers=max_speakers)

config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)


In [11]:
result = whisperx.assign_word_speakers(diarize_segments, result)
print(diarize_segments)
print(result["segments"]) # segments are now assigned speaker IDs

                               segment label     speaker        start  \
0    [ 00:00:06.274 -->  00:00:08.181]     A  SPEAKER_01     6.274719   
1    [ 00:00:08.687 -->  00:00:16.298]     B  SPEAKER_01     8.687844   
2    [ 00:00:16.703 -->  00:00:21.749]     C  SPEAKER_01    16.703469   
3    [ 00:00:21.749 -->  00:00:22.339]     D  SPEAKER_00    21.749094   
4    [ 00:00:22.575 -->  00:00:25.799]     E  SPEAKER_01    22.575969   
..                                 ...   ...         ...          ...   
204  [ 00:18:01.178 -->  00:18:17.159]    GW  SPEAKER_00  1081.178469   
205  [ 00:18:17.175 -->  00:18:22.154]    GX  SPEAKER_01  1097.175969   
206  [ 00:18:17.580 -->  00:18:18.036]    GY  SPEAKER_00  1097.580969   
207  [ 00:18:22.424 -->  00:18:25.495]    GZ  SPEAKER_01  1102.424094   
208  [ 00:18:25.714 -->  00:18:30.338]    HA  SPEAKER_01  1105.714719   

             end  intersection        union  
0       8.181594  -1101.775406  1104.162281  
1      16.298469  -1093.658531 

In [17]:
SAVE_FOLDER = "labeled_voices"

from pydub import AudioSegment

speaker_mapping ={
    "SPEAKER_00": "Cvetanka_Rizova",
    "SPEAKER_01": "Kiril_Petkov",
}

# Load full audio file
audio = AudioSegment.from_file(audio_file)  # Automatically detects format (MP3, WAV, etc.)

# Loop through diarized segments and save each one as a WAV file
for i, segment in enumerate(result["segments"]):
    start_time = segment["start"] * 1000  # Convert to milliseconds
    end_time = segment["end"] * 1000      # Convert to milliseconds
    speaker_id = segment.get("speaker", "unknown")  # Get speaker ID
    speaker = speaker_mapping.get(speaker_id, speaker_id)  # Use mapping if available

    # Extract segment
    segment_audio = audio[start_time:end_time]

    # Define file name
    segment_filename = f"speaker_{speaker}_segment_{i}.wav"

    # Export as WAV
    segment_audio.export(f"{SAVE_FOLDER}/{segment_filename}", format="wav")

    print(f"Saved: {segment_filename}")


Saved: speaker_Kiril_Petkov_segment_0.wav
Saved: speaker_Kiril_Petkov_segment_1.wav
Saved: speaker_Kiril_Petkov_segment_2.wav
Saved: speaker_Kiril_Petkov_segment_3.wav
Saved: speaker_Kiril_Petkov_segment_4.wav
Saved: speaker_Kiril_Petkov_segment_5.wav
Saved: speaker_Cvetanka_Rizova_segment_6.wav
Saved: speaker_Kiril_Petkov_segment_7.wav
Saved: speaker_Kiril_Petkov_segment_8.wav
Saved: speaker_Cvetanka_Rizova_segment_9.wav
Saved: speaker_Cvetanka_Rizova_segment_10.wav
Saved: speaker_Cvetanka_Rizova_segment_11.wav
Saved: speaker_Cvetanka_Rizova_segment_12.wav
Saved: speaker_Kiril_Petkov_segment_13.wav
Saved: speaker_Kiril_Petkov_segment_14.wav
Saved: speaker_Cvetanka_Rizova_segment_15.wav
Saved: speaker_Cvetanka_Rizova_segment_16.wav
Saved: speaker_Cvetanka_Rizova_segment_17.wav
Saved: speaker_Cvetanka_Rizova_segment_18.wav
Saved: speaker_Cvetanka_Rizova_segment_19.wav
Saved: speaker_Cvetanka_Rizova_segment_20.wav
Saved: speaker_Cvetanka_Rizova_segment_21.wav
Saved: speaker_Cvetanka_Rizo

In [18]:

from google.colab import files
import os

dir_to_zip = SAVE_FOLDER #@param {type: "string"}
output_filename = f'{SAVE_FOLDER}.zip' #@param {type: "string"}
delete_dir_after_download = "No"  #@param ['Yes', 'No']

os.system( "zip -r {} {}".format( output_filename , dir_to_zip ) )

if delete_dir_after_download == "Yes":
    os.system( "rm -r {}".format( dir_to_zip ) )

files.download( output_filename )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>